<a href="https://colab.research.google.com/github/lestimpe/SARS-CoV-2-genome/blob/main/Comparing_Wuhan_And_Unknown(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Comparing the Wuhan isolate with two variants**

We will download the genomic sequences from the Wuhan isolate, a delta variant and an unknown that you will classify at the end of this notebook.  First we install Biopython and other packages:

In [ ]:
!pip install biopython
import Bio
from Bio import Entrez
from Bio import SeqIO
from Bio import GenBank
from Bio import Align
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import requests, sys, json, csv

The next code cell downloads the three genomic sequences:

In [ ]:
Entrez.email = 'A.N.Other@example.com'
with Entrez.efetch(
    db="nucleotide", rettype="gb", retmode="text", id="NC_045512.2"
) as handle:
    Wuhan_record = SeqIO.read(handle, "genbank")
print("%s with %i features" % (Wuhan_record.id, len(Wuhan_record.features)))

Entrez.email = 'A.N.Other@example.com'
with Entrez.efetch(
    db="nucleotide", rettype="gb", retmode="text", id="MZ077342.1"
) as handle:
    Delta_record = SeqIO.read(handle, "genbank")
print("%s with %i features" % (Delta_record.id, len(Delta_record.features)))

Entrez.email = 'A.N.Other@example.com'
with Entrez.efetch(
    db="nucleotide", rettype="gb", retmode="text", id="MZ007342.1"
) as handle:
    Unknown_record = SeqIO.read(handle, "genbank")
print("%s with %i features" % (Unknown_record.id, len(Unknown_record.features)))

In a sequence *alignment*, two or more sequences are arranged to line up the corresponding nucleotides or amino acids.  Remember that there are three reading frames per strand. After making sure that the spike sequences are in the same reading frame, we align them using PairwiseAligner.  The next code cell sets up PairwiseAligner.

In [4]:
aligner = Align.PairwiseAligner()
aligner.mode = 'local'
aligner.mismatch_score = -10
aligner.target_internal_open_gap_score = -20.000000
aligner.target_internal_extend_gap_score = -5.00000
aligner.target_left_open_gap_score = -10.000000
aligner.target_left_extend_gap_score = -0.500000
aligner.target_right_open_gap_score = -10.000000
aligner.target_right_extend_gap_score = -0.500000
aligner.query_internal_open_gap_score = -10.000000
aligner.query_internal_extend_gap_score = -0.500000
aligner.query_left_open_gap_score = -10.000000
aligner.query_left_extend_gap_score = -0.500000
aligner.query_right_open_gap_score = -10.000000
aligner.query_right_extend_gap_score = -0.500000

Now we run the aligner.  The output is the amino acid sequence in the single letter code, Wuhan isolate on top.  

In [ ]:
spikeW = Wuhan_record.seq[21562:25384].translate()
spikeD = Delta_record.seq[21525:25427].translate()
spikeU = Unknown_record.seq[21523:25427].translate()
alignmentWvsD_aa = aligner.align(spikeW, spikeD)

print(alignmentWvsD_aa[0])

Notice that the first amino acid is M, or methionine, as expected.  If you scroll all the way to the end of the sequence you will see asterisks, corresponding to the stop codons, the end of translation and the end of the open reading frame.

The missense mutations and gaps are represented by dots and dashes, respectively.  Scrolling back to the beginning, you will see where the program has aligned LIV in the Wuhan isolate with XXX in the Delta.  There is no amino acid X in the single letter code.  This is a region of nine nucleotides in Delta in which the sequencing failed to identify the nucleotides.  The technology is not perfect, and these things happen.  Not knowing what the amino acids are, the aligner program uses X.

Near the beginning of the sequence you will see T (threonine) in the Wuhan isolate paired with R (arginine) in Delta.  The dot indicates a mismatch.  This mutation is represented as T19R, where 19 is the position of the amino acid.   It is a missense mutation. Each variant is defined by its mutations.  There is a list for a few common variants in a document called *Variants_Spike_mutations* that is at github and should be in your Downloads file. If the sequences get out of register due to insertions or deletions, use the Wuhan isolate for the amino acid addresses.


# Assignment

The assignment is to classify the unknown genome.  We need a multiple sequence alignment, which will include the spike protein from all three viral genomes.  Multiple sequence alignment is a more difficult problem for a computer than pairwise alignment, and can't be done through Biopython.  Instead we will prepare a file of the three spike protein sequences, then submit the file to a computer at the European Bioinformatics Institute (EBI) for alignment.  The EBI will use a program called *CLUSTAL* to align the three sequences.

The following code block prints out the file with three spike sequences.  They are in a format called *fasta*, which is widely used for this sort of job.

In [ ]:
spikeW_r = Seq(spikeW)
spikeW_record = SeqRecord(spikeW_r)
spikeW_record.id = "Wuhan"
spikeD_r = Seq(spikeD)
spikeD_record = SeqRecord(spikeD_r)
spikeD_record.id = "Delta"
spikeU_r = Seq(spikeU)
spikeU_record = SeqRecord(spikeU_r)
spikeU_record.id = "Unknown"
altogether = spikeW_record.format("fasta") + spikeD_record.format("fasta") + spikeU_record.format("fasta")
print(altogether)

Execute the next two code blocks to submit the job.  Depending on how busy the EBI computer is, it will take a few seconds to several hours.

In [ ]:
# https://rest.uniprot.org/beta/docs/
WEBSITE_API = "https://rest.uniprot.org/beta"

# Helper function to download data
def get_url(url, **kwargs):
  response = requests.get(url, **kwargs);

  if not response.ok:
    print(response.text)
    response.raise_for_status()
    sys.exit()

  return response

In [ ]:
r = requests.post("https://www.ebi.ac.uk/Tools/services/rest/clustalo/run", data={
    "email": "lestimpe@gmail.com",
    "iterations": 0,
    "outfmt": "clustal_num",
    "order": "aligned",
    "sequence": altogether
})

job_id = r.text
print(job_id)

# get job status
r = get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{job_id}")
print(r.text)

If the status is "RUNNING", click the following code block to check.  When the job is finished run the last code block to display the aligned sequences.

Open the *Variants_Spike_mutations* file (in your Downloads).  Go through the alignment comparing the Wuhan isolate with the unknown.  Write down the mutations using the notation described above, then compare your results with the lists in *Variants_Spike_mutations* to figure out which variant the unknown is.  If you are interested in learning more about the variants look [here](https://www.nytimes.com/interactive/2021/health/coronavirus-variant-tracker.html?searchResultPosition=83).  The article also has some useful diagrams of the genome and pictures of the spike protein.

A sequence may have small differences compared to the defining mutations, yet still be an example of the variant.  Variants are classified using a phylogenetic tree.  All the variants identified as Delta, for example, have a pattern of mutations more similar to other Delta variants than to any other, but all sequences defined as Delta are not identical.

In [ ]:
# get job status
r = get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{job_id}")

print(r.text)

In [ ]:
r = get_url(f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{job_id}/aln-clustal_num")
print(r.text)